### mecab + neologdn

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
! pip install neologdn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 3.4 MB/s 
  Created wheel for neologdn: filename=neologdn-0.5.1-cp38-cp38-linux_x86_64.whl size=178317 sha256=675a407eb2a7c7fef0c89d422d14834e7e0a9077321a2bcc58c388b80d258cb8
  Stored in directory: /root/.cache/pip/wheels/3e/db/10/b3b26caa63c5da86ea3a25043cc4379a66bb3dd30d6f060a37
Successfully built neologdn


In [4]:
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [15]:
import neologdn
import unicodedata
import MeCab
import re

In [9]:
m = MeCab.Tagger()
sample_txt = "彼女はペンパイナッポーアッポーペンと恋ダンスを踊った。"
print("Mecab:\n", m.parse(sample_txt))
# path 指定
path = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
m = MeCab.Tagger(path)
print("Mecab ipadic NEologd:\n",m.parse(sample_txt))

Mecab:
 彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
ペンパイナッポーアッポーペン	名詞,一般,*,*,*,*,*
と	助詞,並立助詞,*,*,*,*,と,ト,ト
恋	名詞,一般,*,*,*,*,恋,コイ,コイ
ダンス	名詞,サ変接続,*,*,*,*,ダンス,ダンス,ダンス
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
踊っ	動詞,自立,*,*,五段・ラ行,連用タ接続,踊る,オドッ,オドッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。
EOS

Mecab ipadic NEologd:
 彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
ペンパイナッポーアッポーペン	名詞,固有名詞,一般,*,*,*,Pen-Pineapple-Apple-Pen,ペンパイナッポーアッポーペン,ペンパイナッポーアッポーペン
と	助詞,並立助詞,*,*,*,*,と,ト,ト
恋ダンス	名詞,固有名詞,一般,*,*,*,恋ダンス,コイダンス,コイダンス
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
踊っ	動詞,自立,*,*,五段・ラ行,連用タ接続,踊る,オドッ,オドッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。
EOS



In [11]:
%cd '/content/drive/MyDrive/grad_comp'

/content/drive/MyDrive/grad_comp


In [24]:
# neologdn + (小文字化) + 数字の置換
def normalize_number(text):
    """
    pattern = r'\d+'
    replacer = re.compile(pattern)
    result = replacer.sub('0', text)
    """
    # 連続した数字を0で置換
    replaced_text = re.sub(r'\d+', '0', text)
    return replaced_text

filename = ['train','dev','test']

for name in filename:
  with open('preprocess/text.prep_' + name + '.txt') as fr,open('preprocess/text.prep02_' + name + '.txt',mode='w') as fw:
    for line in fr:
      line = neologdn.normalize(line)
      line = normalize_number(line)
      fw.write(line)

In [25]:
# src にmecab後のテキストを入れる
filename = ['train','dev','test']
# neologd 対応
path = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
wakati = MeCab.Tagger(f'-Owakati {path}')
for name in filename:
  with open('preprocess/text.prep02_' + name + '.txt') as fr,open('preprocess/mecab_neologd/text.prep02_' + name + '.txt','w') as fw:
    for line in fr:
      line = wakati.parse(line).strip()
      fw.write(line + '\n')

In [18]:
path = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
wakati = MeCab.Tagger(f'-Owakati {path}')
t = '私の名前は安倍晋三です。'
print(wakati.parse(t).strip())

私 の 名前 は 安倍晋三 です 。


In [27]:
# src にmecab後のテキストを入れる
filename = ['train','dev','test']
wakati = MeCab.Tagger('-Owakati')
for name in filename:
  with open('preprocess/text.prep02_' + name + '.txt') as fr,open('preprocess/mecab/text.prep02_' + name + '.txt','w') as fw:
    for line in fr:
      line = wakati.parse(line).strip()
      fw.write(line + '\n')

## sudachi

In [28]:
# インストール

! pip install sudachipy
! pip install sudachidict_core


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 6.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sudachidict-core: filename=SudachiDict_core-20221021-py3-none-any.whl size=71574782 sha256=c8bca6dee4e46773d1df8875085fdc468fc1f1e82037ae4d70bce3b24a65057a
  Stored in directory: /root/.cache/pip/wheels/ca/e3/ed/e78fecf6fd34349114d292242a16fc08d513fb32c2d9c5d786
Successfully built sudachidict-core


In [31]:
from sudachipy import Dictionary
from sudachipy import SplitMode

tokenizer = Dictionary().create()

SM = SplitMode.C
filename = ['train','dev','test']

for name in filename:
  # 分かち書き + 正規化 したのを上書き
    with open('preprocess/text.prep02_' + name + '.txt') as fr,open('preprocess/sudachi/C/text.prep02_' + name + '.txt','w') as fw:
        for line in fr:
            token_list = list()
            for token in tokenizer.tokenize(line.strip(), SM):
                # 正規化
                token_list.append(token.normalized_form())
            fw.write(' '.join(token_list) + '\n')